## Librerias

In [124]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate,cross_val_score,train_test_split, GridSearchCV
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

import os

import joblib
import parselmouth
from parselmouth.praat import call
from scipy.io import wavfile
from pyrpde import  rpde
import re
import numpy as np
import IPython.display as ipd
import librosa
import noisereduce as nr

## Resultados pacientes con parkinson

In [214]:
features = ["subject#","age","sex","test_time","motor_UPDRS","total_UPDRS","Jitter(%)","Jitter(Abs)","Jitter:RAP","Jitter:PPQ5","Jitter:DDP","Shimmer","Shimmer(dB)","Shimmer:APQ3","Shimmer:APQ5","Shimmer:APQ11","Shimmer:DDA","NHR","HNR","RPDE","DFA","PPE"]
dataset = pd.read_csv("./data/parkinsons_updrs.data",names=features)
# df = dataset.drop(['name','status'], axis=1)
df = dataset.drop(["subject#","age","sex","test_time","motor_UPDRS","total_UPDRS","DFA","PPE"], axis=1)
df = df.drop(index=0).reset_index(drop=True)
df = df.rename(columns={
    "Jitter(%)": "MDVP:Jitter(%)",
    "Jitter(Abs)": "MDVP:Jitter(Abs)",
    "Jitter:RAP": "MDVP:RAP",
    "Jitter:PPQ5": "MDVP:PPQ",
    "Shimmer": "MDVP:Shimmer",
    "Shimmer(dB)": "MDVP:Shimmer(dB)",
    "Shimmer:APQ11": "MDVP:APQ"
})
ordered_columns = [
    "MDVP:Jitter(%)", "MDVP:Jitter(Abs)", "MDVP:RAP", "MDVP:PPQ", "Jitter:DDP",
    "MDVP:Shimmer", "MDVP:Shimmer(dB)", "Shimmer:APQ3", "Shimmer:APQ5",
    "MDVP:APQ", "Shimmer:DDA", "NHR", "HNR"
]
df = df[ordered_columns]
df["status"] = 1
# df = df.head(762)
df = df.head(2000)
df.head()

,MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status
0,0.00662,3.38e-005,0.00401,0.00317,0.01204,0.02565,0.23,0.01438,0.01309,0.01662,0.04314,0.01429,21.64,1
1,0.003,1.68e-005,0.00132,0.0015,0.00395,0.02024,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,1
2,0.00481,2.462e-005,0.00205,0.00208,0.00616,0.01675,0.181,0.00734,0.00844,0.01458,0.02202,0.02022,23.047,1
3,0.00528,2.657e-005,0.00191,0.00264,0.00573,0.02309,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,1
4,0.00335,2.014e-005,0.00093,0.0013,0.00278,0.01703,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,1


In [6]:
features = ["Filename","MDVP:Fo(Hz)","MDVP:Fhi(Hz)","MDVP:Flo(Hz)","MDVP:Jitter(%)","MDVP:Jitter(Abs)","MDVP:RAP","MDVP:PPQ","Jitter:DDP","MDVP:Shimmer","MDVP:Shimmer(dB)","Shimmer:APQ3","Shimmer:APQ5","MDVP:APQ","Shimmer:DDA","NHR","HNR"]
dataset = pd.read_csv("./data/extracted_features.csv",names=features)
# df = dataset.drop(['name','status'], axis=1)
df1 = dataset.drop(["Filename","MDVP:Fo(Hz)","MDVP:Fhi(Hz)","MDVP:Flo(Hz)"], axis=1)
df1 = df1.drop(index=0).reset_index(drop=True)
df1.head()

,MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR
0,0.005366925641098722,3.434705487559645e-05,0.0031725984099005918,0.0029010868949849436,0.009517795229701776,0.08236143798540542,0.7577522933751293,0.04439570958500341,0.05831743853320764,0.05338392982666483,0.13318712875501024,0.028628,17.064
1,0.004274469555576444,2.772530573527911e-05,0.002423545234707254,0.002411515460443344,0.007270635704121762,0.04204661470600538,0.3891155652751097,0.022667580308055955,0.02966768858629313,0.02906772968429414,0.06800274092416786,0.012274,20.464
2,0.007534380817536875,4.932693196532115e-05,0.004691829210163016,0.003920275777606227,0.014075487630489047,0.06158906827924478,0.5644331409911827,0.033166080536055256,0.0445340022285507,0.04047554909071743,0.09949824160816577,0.023968,17.402
3,0.00695140706754981,4.0846710318540045e-05,0.004298470693462664,0.0037802080112414923,0.01289541208038799,0.06808745521225981,0.648251557546415,0.03599204270840177,0.05154802023386642,0.04614895484099407,0.10797612812520531,0.023081,17.242
4,0.005705620089053313,3.2110995241214395e-05,0.0035487548036761594,0.003065328639840235,0.010646264411028478,0.08270264917626552,0.7626035291957517,0.0463550597632467,0.0555741539007348,0.05241792132265787,0.1390651792897401,0.019840,18.038


In [7]:
features = ["name", "MDVP:Fo(Hz)", "MDVP:Fhi(Hz)", "MDVP:Flo(Hz)", "MDVP:Jitter(%)", 
            "MDVP:Jitter(Abs)", "MDVP:RAP", "MDVP:PPQ", "Jitter:DDP", "MDVP:Shimmer", 
            "MDVP:Shimmer(dB)", "Shimmer:APQ3", "Shimmer:APQ5", "MDVP:APQ", "Shimmer:DDA", 
            "NHR", "HNR", "status", "RPDE", "DFA", "spread1", "spread2", "D2", "PPE"]

# Cargar el dataset con nombres de columnas
dataset = pd.read_csv("./data/parkinsons.data", names=features)
dataset["status"] = pd.to_numeric(dataset["status"], errors='coerce').fillna(0).astype(int)

# Filtrar filas donde status es 0
df2 = dataset[dataset["status"] == 0]
ordered_columns = ["MDVP:Jitter(%)", "MDVP:Jitter(Abs)", "MDVP:RAP", "MDVP:PPQ", "Jitter:DDP",
                   "MDVP:Shimmer", "MDVP:Shimmer(dB)", "Shimmer:APQ3", "Shimmer:APQ5", 
                   "MDVP:APQ", "Shimmer:DDA", "NHR", "HNR"]
df2 = df2[ordered_columns]
df2 = df2.drop(index=0).reset_index(drop=True)

df2.head()


,MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR
0,0.00289,0.00001,0.00166,0.00168,0.00498,0.01098,0.09700,0.00563,0.00680,0.00802,0.01689,0.00339,26.77500
1,0.00241,0.00001,0.00134,0.00138,0.00402,0.01015,0.08900,0.00504,0.00641,0.00762,0.01513,0.00167,30.94000
2,0.00212,0.00001,0.00113,0.00135,0.00339,0.01263,0.11100,0.00640,0.00825,0.00951,0.01919,0.00119,30.77500
3,0.00180,0.000009,0.00093,0.00107,0.00278,0.00954,0.08500,0.00469,0.00606,0.00719,0.01407,0.00072,32.68400
4,0.00178,0.000009,0.00094,0.00106,0.00283,0.00958,0.08500,0.00468,0.00610,0.00726,0.01403,0.00065,33.04700


In [8]:
columns = [
  "id","gender","PPE","DFA","RPDE","numPulses","numPeriodsPulses","meanPeriodPulses","stdDevPeriodPulses","locPctJitter","locAbsJitter","rapJitter","ppq5Jitter","ddpJitter","locShimmer","locDbShimmer","apq3Shimmer","apq5Shimmer","apq11Shimmer","ddaShimmer","meanAutoCorrHarmonicity","meanNoiseToHarmHarmonicity","meanHarmToNoiseHarmonicity","minIntensity","maxIntensity","meanIntensity","f1","f2","f3","f4","b1","b2","b3","b4","GQ_prc5_95","GQ_std_cycle_open","GQ_std_cycle_closed","GNE_mean","GNE_std","GNE_SNR_TKEO","GNE_SNR_SEO","GNE_NSR_TKEO","GNE_NSR_SEO","VFER_mean","VFER_std","VFER_entropy","VFER_SNR_TKEO","VFER_SNR_SEO","VFER_NSR_TKEO","VFER_NSR_SEO","IMF_SNR_SEO","IMF_SNR_TKEO","IMF_SNR_entropy","IMF_NSR_SEO","IMF_NSR_TKEO","IMF_NSR_entropy","mean_Log_energy","mean_MFCC_0th_coef","mean_MFCC_1st_coef","mean_MFCC_2nd_coef","mean_MFCC_3rd_coef","mean_MFCC_4th_coef","mean_MFCC_5th_coef","mean_MFCC_6th_coef","mean_MFCC_7th_coef","mean_MFCC_8th_coef","mean_MFCC_9th_coef","mean_MFCC_10th_coef","mean_MFCC_11th_coef","mean_MFCC_12th_coef","mean_delta_log_energy","mean_0th_delta","mean_1st_delta","mean_2nd_delta","mean_3rd_delta","mean_4th_delta","mean_5th_delta","mean_6th_delta","mean_7th_delta","mean_8th_delta","mean_9th_delta","mean_10th_delta","mean_11th_delta","mean_12th_delta","mean_delta_delta_log_energy","mean_delta_delta_0th","mean_1st_delta_delta","mean_2nd_delta_delta","mean_3rd_delta_delta","mean_4th_delta_delta","mean_5th_delta_delta","mean_6th_delta_delta","mean_7th_delta_delta","mean_8th_delta_delta","mean_9th_delta_delta","mean_10th_delta_delta","mean_11th_delta_delta","mean_12th_delta_delta","std_Log_energy","std_MFCC_0th_coef","std_MFCC_1st_coef","std_MFCC_2nd_coef","std_MFCC_3rd_coef","std_MFCC_4th_coef","std_MFCC_5th_coef","std_MFCC_6th_coef","std_MFCC_7th_coef","std_MFCC_8th_coef","std_MFCC_9th_coef","std_MFCC_10th_coef","std_MFCC_11th_coef","std_MFCC_12th_coef","std_delta_log_energy","std_0th_delta","std_1st_delta","std_2nd_delta","std_3rd_delta","std_4th_delta","std_5th_delta","std_6th_delta","std_7th_delta","std_8th_delta","std_9th_delta","std_10th_delta","std_11th_delta","std_12th_delta","std_delta_delta_log_energy","std_delta_delta_0th","std_1st_delta_delta","std_2nd_delta_delta","std_3rd_delta_delta","std_4th_delta_delta","std_5th_delta_delta","std_6th_delta_delta","std_7th_delta_delta","std_8th_delta_delta","std_9th_delta_delta","std_10th_delta_delta","std_11th_delta_delta","std_12th_delta_delta","Ea","Ed_1_coef","Ed_2_coef","Ed_3_coef","Ed_4_coef","Ed_5_coef","Ed_6_coef","Ed_7_coef","Ed_8_coef","Ed_9_coef","Ed_10_coef","det_entropy_shannon_1_coef","det_entropy_shannon_2_coef","det_entropy_shannon_3_coef","det_entropy_shannon_4_coef","det_entropy_shannon_5_coef","det_entropy_shannon_6_coef","det_entropy_shannon_7_coef","det_entropy_shannon_8_coef","det_entropy_shannon_9_coef","det_entropy_shannon_10_coef","det_entropy_log_1_coef","det_entropy_log_2_coef","det_entropy_log_3_coef","det_entropy_log_4_coef","det_entropy_log_5_coef","det_entropy_log_6_coef","det_entropy_log_7_coef","det_entropy_log_8_coef","det_entropy_log_9_coef","det_entropy_log_10_coef","det_TKEO_mean_1_coef","det_TKEO_mean_2_coef","det_TKEO_mean_3_coef","det_TKEO_mean_4_coef","det_TKEO_mean_5_coef","det_TKEO_mean_6_coef","det_TKEO_mean_7_coef","det_TKEO_mean_8_coef","det_TKEO_mean_9_coef","det_TKEO_mean_10_coef","det_TKEO_std_1_coef","det_TKEO_std_2_coef","det_TKEO_std_3_coef","det_TKEO_std_4_coef","det_TKEO_std_5_coef","det_TKEO_std_6_coef","det_TKEO_std_7_coef","det_TKEO_std_8_coef","det_TKEO_std_9_coef","det_TKEO_std_10_coef","app_entropy_shannon_1_coef","app_entropy_shannon_2_coef","app_entropy_shannon_3_coef","app_entropy_shannon_4_coef","app_entropy_shannon_5_coef","app_entropy_shannon_6_coef","app_entropy_shannon_7_coef","app_entropy_shannon_8_coef","app_entropy_shannon_9_coef","app_entropy_shannon_10_coef","app_entropy_log_1_coef","app_entropy_log_2_coef","app_entropy_log_3_coef","app_entropy_log_4_coef","app_entropy_log_5_coef","app_entropy_log_6_coef","app_entropy_log_7_coef","app_entropy_log_8_coef","app_entropy_log_9_coef","app_entropy_log_10_coef","app_det_TKEO_mean_1_coef","app_det_TKEO_mean_2_coef","app_det_TKEO_mean_3_coef","app_det_TKEO_mean_4_coef","app_det_TKEO_mean_5_coef","app_det_TKEO_mean_6_coef","app_det_TKEO_mean_7_coef","app_det_TKEO_mean_8_coef","app_det_TKEO_mean_9_coef","app_det_TKEO_mean_10_coef","app_TKEO_std_1_coef","app_TKEO_std_2_coef","app_TKEO_std_3_coef","app_TKEO_std_4_coef","app_TKEO_std_5_coef","app_TKEO_std_6_coef","app_TKEO_std_7_coef","app_TKEO_std_8_coef","app_TKEO_std_9_coef","app_TKEO_std_10_coef","Ea2","Ed2_1_coef","Ed2_2_coef","Ed2_3_coef","Ed2_4_coef","Ed2_5_coef","Ed2_6_coef","Ed2_7_coef","Ed2_8_coef","Ed2_9_coef","Ed2_10_coef","det_LT_entropy_shannon_1_coef","det_LT_entropy_shannon_2_coef","det_LT_entropy_shannon_3_coef","det_LT_entropy_shannon_4_coef","det_LT_entropy_shannon_5_coef","det_LT_entropy_shannon_6_coef","det_LT_entropy_shannon_7_coef","det_LT_entropy_shannon_8_coef","det_LT_entropy_shannon_9_coef","det_LT_entropy_shannon_10_coef","det_LT_entropy_log_1_coef","det_LT_entropy_log_2_coef","det_LT_entropy_log_3_coef","det_LT_entropy_log_4_coef","det_LT_entropy_log_5_coef","det_LT_entropy_log_6_coef","det_LT_entropy_log_7_coef","det_LT_entropy_log_8_coef","det_LT_entropy_log_9_coef","det_LT_entropy_log_10_coef","det_LT_TKEO_mean_1_coef","det_LT_TKEO_mean_2_coef","det_LT_TKEO_mean_3_coef","det_LT_TKEO_mean_4_coef","det_LT_TKEO_mean_5_coef","det_LT_TKEO_mean_6_coef","det_LT_TKEO_mean_7_coef","det_LT_TKEO_mean_8_coef","det_LT_TKEO_mean_9_coef","det_LT_TKEO_mean_10_coef","det_LT_TKEO_std_1_coef","det_LT_TKEO_std_2_coef","det_LT_TKEO_std_3_coef","det_LT_TKEO_std_4_coef","det_LT_TKEO_std_5_coef","det_LT_TKEO_std_6_coef","det_LT_TKEO_std_7_coef","det_LT_TKEO_std_8_coef","det_LT_TKEO_std_9_coef","det_LT_TKEO_std_10_coef","app_LT_entropy_shannon_1_coef","app_LT_entropy_shannon_2_coef","app_LT_entropy_shannon_3_coef","app_LT_entropy_shannon_4_coef","app_LT_entropy_shannon_5_coef","app_LT_entropy_shannon_6_coef","app_LT_entropy_shannon_7_coef","app_LT_entropy_shannon_8_coef","app_LT_entropy_shannon_9_coef","app_LT_entropy_shannon_10_coef","app_LT_entropy_log_1_coef","app_LT_entropy_log_2_coef","app_LT_entropy_log_3_coef","app_LT_entropy_log_4_coef","app_LT_entropy_log_5_coef","app_LT_entropy_log_6_coef","app_LT_entropy_log_7_coef","app_LT_entropy_log_8_coef","app_LT_entropy_log_9_coef","app_LT_entropy_log_10_coef","app_LT_TKEO_mean_1_coef","app_LT_TKEO_mean_2_coef","app_LT_TKEO_mean_3_coef","app_LT_TKEO_mean_4_coef","app_LT_TKEO_mean_5_coef","app_LT_TKEO_mean_6_coef","app_LT_TKEO_mean_7_coef","app_LT_TKEO_mean_8_coef","app_LT_TKEO_mean_9_coef","app_LT_TKEO_mean_10_coef","app_LT_TKEO_std_1_coef","app_LT_TKEO_std_2_coef","app_LT_TKEO_std_3_coef","app_LT_TKEO_std_4_coef","app_LT_TKEO_std_5_coef","app_LT_TKEO_std_6_coef","app_LT_TKEO_std_7_coef","app_LT_TKEO_std_8_coef","app_LT_TKEO_std_9_coef","app_LT_TKEO_std_10_coef","tqwt_energy_dec_1","tqwt_energy_dec_2","tqwt_energy_dec_3","tqwt_energy_dec_4","tqwt_energy_dec_5","tqwt_energy_dec_6","tqwt_energy_dec_7","tqwt_energy_dec_8","tqwt_energy_dec_9","tqwt_energy_dec_10","tqwt_energy_dec_11","tqwt_energy_dec_12","tqwt_energy_dec_13","tqwt_energy_dec_14","tqwt_energy_dec_15","tqwt_energy_dec_16","tqwt_energy_dec_17","tqwt_energy_dec_18","tqwt_energy_dec_19","tqwt_energy_dec_20","tqwt_energy_dec_21","tqwt_energy_dec_22","tqwt_energy_dec_23","tqwt_energy_dec_24","tqwt_energy_dec_25","tqwt_energy_dec_26","tqwt_energy_dec_27","tqwt_energy_dec_28","tqwt_energy_dec_29","tqwt_energy_dec_30","tqwt_energy_dec_31","tqwt_energy_dec_32","tqwt_energy_dec_33","tqwt_energy_dec_34","tqwt_energy_dec_35","tqwt_energy_dec_36","tqwt_entropy_shannon_dec_1","tqwt_entropy_shannon_dec_2","tqwt_entropy_shannon_dec_3","tqwt_entropy_shannon_dec_4","tqwt_entropy_shannon_dec_5","tqwt_entropy_shannon_dec_6","tqwt_entropy_shannon_dec_7","tqwt_entropy_shannon_dec_8","tqwt_entropy_shannon_dec_9","tqwt_entropy_shannon_dec_10","tqwt_entropy_shannon_dec_11","tqwt_entropy_shannon_dec_12","tqwt_entropy_shannon_dec_13","tqwt_entropy_shannon_dec_14","tqwt_entropy_shannon_dec_15","tqwt_entropy_shannon_dec_16","tqwt_entropy_shannon_dec_17","tqwt_entropy_shannon_dec_18","tqwt_entropy_shannon_dec_19","tqwt_entropy_shannon_dec_20","tqwt_entropy_shannon_dec_21","tqwt_entropy_shannon_dec_22","tqwt_entropy_shannon_dec_23","tqwt_entropy_shannon_dec_24","tqwt_entropy_shannon_dec_25","tqwt_entropy_shannon_dec_26","tqwt_entropy_shannon_dec_27","tqwt_entropy_shannon_dec_28","tqwt_entropy_shannon_dec_29","tqwt_entropy_shannon_dec_30","tqwt_entropy_shannon_dec_31","tqwt_entropy_shannon_dec_32","tqwt_entropy_shannon_dec_33","tqwt_entropy_shannon_dec_34","tqwt_entropy_shannon_dec_35","tqwt_entropy_shannon_dec_36","tqwt_entropy_log_dec_1","tqwt_entropy_log_dec_2","tqwt_entropy_log_dec_3","tqwt_entropy_log_dec_4","tqwt_entropy_log_dec_5","tqwt_entropy_log_dec_6","tqwt_entropy_log_dec_7","tqwt_entropy_log_dec_8","tqwt_entropy_log_dec_9","tqwt_entropy_log_dec_10","tqwt_entropy_log_dec_11","tqwt_entropy_log_dec_12","tqwt_entropy_log_dec_13","tqwt_entropy_log_dec_14","tqwt_entropy_log_dec_15","tqwt_entropy_log_dec_16","tqwt_entropy_log_dec_17","tqwt_entropy_log_dec_18","tqwt_entropy_log_dec_19","tqwt_entropy_log_dec_20","tqwt_entropy_log_dec_21","tqwt_entropy_log_dec_22","tqwt_entropy_log_dec_23","tqwt_entropy_log_dec_24","tqwt_entropy_log_dec_25","tqwt_entropy_log_dec_26","tqwt_entropy_log_dec_27","tqwt_entropy_log_dec_28","tqwt_entropy_log_dec_29","tqwt_entropy_log_dec_30","tqwt_entropy_log_dec_31","tqwt_entropy_log_dec_32","tqwt_entropy_log_dec_33","tqwt_entropy_log_dec_34","tqwt_entropy_log_dec_35","tqwt_entropy_log_dec_36","tqwt_TKEO_mean_dec_1","tqwt_TKEO_mean_dec_2","tqwt_TKEO_mean_dec_3","tqwt_TKEO_mean_dec_4","tqwt_TKEO_mean_dec_5","tqwt_TKEO_mean_dec_6","tqwt_TKEO_mean_dec_7","tqwt_TKEO_mean_dec_8","tqwt_TKEO_mean_dec_9","tqwt_TKEO_mean_dec_10","tqwt_TKEO_mean_dec_11","tqwt_TKEO_mean_dec_12","tqwt_TKEO_mean_dec_13","tqwt_TKEO_mean_dec_14","tqwt_TKEO_mean_dec_15","tqwt_TKEO_mean_dec_16","tqwt_TKEO_mean_dec_17","tqwt_TKEO_mean_dec_18","tqwt_TKEO_mean_dec_19","tqwt_TKEO_mean_dec_20","tqwt_TKEO_mean_dec_21","tqwt_TKEO_mean_dec_22","tqwt_TKEO_mean_dec_23","tqwt_TKEO_mean_dec_24","tqwt_TKEO_mean_dec_25","tqwt_TKEO_mean_dec_26","tqwt_TKEO_mean_dec_27","tqwt_TKEO_mean_dec_28","tqwt_TKEO_mean_dec_29","tqwt_TKEO_mean_dec_30","tqwt_TKEO_mean_dec_31","tqwt_TKEO_mean_dec_32","tqwt_TKEO_mean_dec_33","tqwt_TKEO_mean_dec_34","tqwt_TKEO_mean_dec_35","tqwt_TKEO_mean_dec_36","tqwt_TKEO_std_dec_1","tqwt_TKEO_std_dec_2","tqwt_TKEO_std_dec_3","tqwt_TKEO_std_dec_4","tqwt_TKEO_std_dec_5","tqwt_TKEO_std_dec_6","tqwt_TKEO_std_dec_7","tqwt_TKEO_std_dec_8","tqwt_TKEO_std_dec_9","tqwt_TKEO_std_dec_10","tqwt_TKEO_std_dec_11","tqwt_TKEO_std_dec_12","tqwt_TKEO_std_dec_13","tqwt_TKEO_std_dec_14","tqwt_TKEO_std_dec_15","tqwt_TKEO_std_dec_16","tqwt_TKEO_std_dec_17","tqwt_TKEO_std_dec_18","tqwt_TKEO_std_dec_19","tqwt_TKEO_std_dec_20","tqwt_TKEO_std_dec_21","tqwt_TKEO_std_dec_22","tqwt_TKEO_std_dec_23","tqwt_TKEO_std_dec_24","tqwt_TKEO_std_dec_25","tqwt_TKEO_std_dec_26","tqwt_TKEO_std_dec_27","tqwt_TKEO_std_dec_28","tqwt_TKEO_std_dec_29","tqwt_TKEO_std_dec_30","tqwt_TKEO_std_dec_31","tqwt_TKEO_std_dec_32","tqwt_TKEO_std_dec_33","tqwt_TKEO_std_dec_34","tqwt_TKEO_std_dec_35","tqwt_TKEO_std_dec_36","tqwt_medianValue_dec_1","tqwt_medianValue_dec_2","tqwt_medianValue_dec_3","tqwt_medianValue_dec_4","tqwt_medianValue_dec_5","tqwt_medianValue_dec_6","tqwt_medianValue_dec_7","tqwt_medianValue_dec_8","tqwt_medianValue_dec_9","tqwt_medianValue_dec_10","tqwt_medianValue_dec_11","tqwt_medianValue_dec_12","tqwt_medianValue_dec_13","tqwt_medianValue_dec_14","tqwt_medianValue_dec_15","tqwt_medianValue_dec_16","tqwt_medianValue_dec_17","tqwt_medianValue_dec_18","tqwt_medianValue_dec_19","tqwt_medianValue_dec_20","tqwt_medianValue_dec_21","tqwt_medianValue_dec_22","tqwt_medianValue_dec_23","tqwt_medianValue_dec_24","tqwt_medianValue_dec_25","tqwt_medianValue_dec_26","tqwt_medianValue_dec_27","tqwt_medianValue_dec_28","tqwt_medianValue_dec_29","tqwt_medianValue_dec_30","tqwt_medianValue_dec_31","tqwt_medianValue_dec_32","tqwt_medianValue_dec_33","tqwt_medianValue_dec_34","tqwt_medianValue_dec_35","tqwt_medianValue_dec_36","tqwt_meanValue_dec_1","tqwt_meanValue_dec_2","tqwt_meanValue_dec_3","tqwt_meanValue_dec_4","tqwt_meanValue_dec_5","tqwt_meanValue_dec_6","tqwt_meanValue_dec_7","tqwt_meanValue_dec_8","tqwt_meanValue_dec_9","tqwt_meanValue_dec_10","tqwt_meanValue_dec_11","tqwt_meanValue_dec_12","tqwt_meanValue_dec_13","tqwt_meanValue_dec_14","tqwt_meanValue_dec_15","tqwt_meanValue_dec_16","tqwt_meanValue_dec_17","tqwt_meanValue_dec_18","tqwt_meanValue_dec_19","tqwt_meanValue_dec_20","tqwt_meanValue_dec_21","tqwt_meanValue_dec_22","tqwt_meanValue_dec_23","tqwt_meanValue_dec_24","tqwt_meanValue_dec_25","tqwt_meanValue_dec_26","tqwt_meanValue_dec_27","tqwt_meanValue_dec_28","tqwt_meanValue_dec_29","tqwt_meanValue_dec_30","tqwt_meanValue_dec_31","tqwt_meanValue_dec_32","tqwt_meanValue_dec_33","tqwt_meanValue_dec_34","tqwt_meanValue_dec_35","tqwt_meanValue_dec_36","tqwt_stdValue_dec_1","tqwt_stdValue_dec_2","tqwt_stdValue_dec_3","tqwt_stdValue_dec_4","tqwt_stdValue_dec_5","tqwt_stdValue_dec_6","tqwt_stdValue_dec_7","tqwt_stdValue_dec_8","tqwt_stdValue_dec_9","tqwt_stdValue_dec_10","tqwt_stdValue_dec_11","tqwt_stdValue_dec_12","tqwt_stdValue_dec_13","tqwt_stdValue_dec_14","tqwt_stdValue_dec_15","tqwt_stdValue_dec_16","tqwt_stdValue_dec_17","tqwt_stdValue_dec_18","tqwt_stdValue_dec_19","tqwt_stdValue_dec_20","tqwt_stdValue_dec_21","tqwt_stdValue_dec_22","tqwt_stdValue_dec_23","tqwt_stdValue_dec_24","tqwt_stdValue_dec_25","tqwt_stdValue_dec_26","tqwt_stdValue_dec_27","tqwt_stdValue_dec_28","tqwt_stdValue_dec_29","tqwt_stdValue_dec_30","tqwt_stdValue_dec_31","tqwt_stdValue_dec_32","tqwt_stdValue_dec_33","tqwt_stdValue_dec_34","tqwt_stdValue_dec_35","tqwt_stdValue_dec_36","tqwt_minValue_dec_1","tqwt_minValue_dec_2","tqwt_minValue_dec_3","tqwt_minValue_dec_4","tqwt_minValue_dec_5","tqwt_minValue_dec_6","tqwt_minValue_dec_7","tqwt_minValue_dec_8","tqwt_minValue_dec_9","tqwt_minValue_dec_10","tqwt_minValue_dec_11","tqwt_minValue_dec_12","tqwt_minValue_dec_13","tqwt_minValue_dec_14","tqwt_minValue_dec_15","tqwt_minValue_dec_16","tqwt_minValue_dec_17","tqwt_minValue_dec_18","tqwt_minValue_dec_19","tqwt_minValue_dec_20","tqwt_minValue_dec_21","tqwt_minValue_dec_22","tqwt_minValue_dec_23","tqwt_minValue_dec_24","tqwt_minValue_dec_25","tqwt_minValue_dec_26","tqwt_minValue_dec_27","tqwt_minValue_dec_28","tqwt_minValue_dec_29","tqwt_minValue_dec_30","tqwt_minValue_dec_31","tqwt_minValue_dec_32","tqwt_minValue_dec_33","tqwt_minValue_dec_34","tqwt_minValue_dec_35","tqwt_minValue_dec_36","tqwt_maxValue_dec_1","tqwt_maxValue_dec_2","tqwt_maxValue_dec_3","tqwt_maxValue_dec_4","tqwt_maxValue_dec_5","tqwt_maxValue_dec_6","tqwt_maxValue_dec_7","tqwt_maxValue_dec_8","tqwt_maxValue_dec_9","tqwt_maxValue_dec_10","tqwt_maxValue_dec_11","tqwt_maxValue_dec_12","tqwt_maxValue_dec_13","tqwt_maxValue_dec_14","tqwt_maxValue_dec_15","tqwt_maxValue_dec_16","tqwt_maxValue_dec_17","tqwt_maxValue_dec_18","tqwt_maxValue_dec_19","tqwt_maxValue_dec_20","tqwt_maxValue_dec_21","tqwt_maxValue_dec_22","tqwt_maxValue_dec_23","tqwt_maxValue_dec_24","tqwt_maxValue_dec_25","tqwt_maxValue_dec_26","tqwt_maxValue_dec_27","tqwt_maxValue_dec_28","tqwt_maxValue_dec_29","tqwt_maxValue_dec_30","tqwt_maxValue_dec_31","tqwt_maxValue_dec_32","tqwt_maxValue_dec_33","tqwt_maxValue_dec_34","tqwt_maxValue_dec_35","tqwt_maxValue_dec_36","tqwt_skewnessValue_dec_1","tqwt_skewnessValue_dec_2","tqwt_skewnessValue_dec_3","tqwt_skewnessValue_dec_4","tqwt_skewnessValue_dec_5","tqwt_skewnessValue_dec_6","tqwt_skewnessValue_dec_7","tqwt_skewnessValue_dec_8","tqwt_skewnessValue_dec_9","tqwt_skewnessValue_dec_10","tqwt_skewnessValue_dec_11","tqwt_skewnessValue_dec_12","tqwt_skewnessValue_dec_13","tqwt_skewnessValue_dec_14","tqwt_skewnessValue_dec_15","tqwt_skewnessValue_dec_16","tqwt_skewnessValue_dec_17","tqwt_skewnessValue_dec_18","tqwt_skewnessValue_dec_19","tqwt_skewnessValue_dec_20","tqwt_skewnessValue_dec_21","tqwt_skewnessValue_dec_22","tqwt_skewnessValue_dec_23","tqwt_skewnessValue_dec_24","tqwt_skewnessValue_dec_25","tqwt_skewnessValue_dec_26","tqwt_skewnessValue_dec_27","tqwt_skewnessValue_dec_28","tqwt_skewnessValue_dec_29","tqwt_skewnessValue_dec_30","tqwt_skewnessValue_dec_31","tqwt_skewnessValue_dec_32","tqwt_skewnessValue_dec_33","tqwt_skewnessValue_dec_34","tqwt_skewnessValue_dec_35","tqwt_skewnessValue_dec_36","tqwt_kurtosisValue_dec_1","tqwt_kurtosisValue_dec_2","tqwt_kurtosisValue_dec_3","tqwt_kurtosisValue_dec_4","tqwt_kurtosisValue_dec_5","tqwt_kurtosisValue_dec_6","tqwt_kurtosisValue_dec_7","tqwt_kurtosisValue_dec_8","tqwt_kurtosisValue_dec_9","tqwt_kurtosisValue_dec_10","tqwt_kurtosisValue_dec_11","tqwt_kurtosisValue_dec_12","tqwt_kurtosisValue_dec_13","tqwt_kurtosisValue_dec_14","tqwt_kurtosisValue_dec_15","tqwt_kurtosisValue_dec_16","tqwt_kurtosisValue_dec_17","tqwt_kurtosisValue_dec_18","tqwt_kurtosisValue_dec_19","tqwt_kurtosisValue_dec_20","tqwt_kurtosisValue_dec_21","tqwt_kurtosisValue_dec_22","tqwt_kurtosisValue_dec_23","tqwt_kurtosisValue_dec_24","tqwt_kurtosisValue_dec_25","tqwt_kurtosisValue_dec_26","tqwt_kurtosisValue_dec_27","tqwt_kurtosisValue_dec_28","tqwt_kurtosisValue_dec_29","tqwt_kurtosisValue_dec_30","tqwt_kurtosisValue_dec_31","tqwt_kurtosisValue_dec_32","tqwt_kurtosisValue_dec_33","tqwt_kurtosisValue_dec_34","tqwt_kurtosisValue_dec_35","tqwt_kurtosisValue_dec_36","class"
]
dataset = pd.read_csv("./data/pd_speech_features.csv",names=columns)
selected_columns = ["locPctJitter", "locAbsJitter", "rapJitter", "ppq5Jitter", "ddpJitter", 
                    "locShimmer", "locDbShimmer", "apq3Shimmer", "apq5Shimmer", 
                    "apq11Shimmer", "ddaShimmer", "meanNoiseToHarmHarmonicity", 
                    "meanHarmToNoiseHarmonicity"]
dataset["class"] = pd.to_numeric(dataset["class"], errors='coerce').fillna(0).astype(int)
df3 = dataset[dataset["class"] == 0]
df3 = df3[selected_columns]
df3 = df3.rename(columns={
    "locPctJitter": "MDVP:Jitter(%)",
    "locAbsJitter": "MDVP:Jitter(Abs)",
    "rapJitter": "MDVP:RAP",
    "ppq5Jitter": "MDVP:PPQ",
    "ddpJitter": "Jitter:DDP",
    "locShimmer": "MDVP:Shimmer",
    "locDbShimmer": "MDVP:Shimmer(dB)",
    "apq3Shimmer": "Shimmer:APQ3",
    "apq5Shimmer": "Shimmer:APQ5",
    "apq11Shimmer": "MDVP:APQ",
    "ddaShimmer": "Shimmer:DDA",
    "meanNoiseToHarmHarmonicity": "NHR",
    "meanHarmToNoiseHarmonicity": "HNR"
})
ordered_columns = [
    "MDVP:Jitter(%)", "MDVP:Jitter(Abs)", "MDVP:RAP", "MDVP:PPQ", "Jitter:DDP",
    "MDVP:Shimmer", "MDVP:Shimmer(dB)", "Shimmer:APQ3", "Shimmer:APQ5",
    "MDVP:APQ", "Shimmer:DDA", "NHR", "HNR"
]
df3 = df3.drop(index=0).reset_index(drop=True)
df3 = df3[ordered_columns]
df3.head()

,MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR
0,0.00086,4.40E-06,9.00E-05,0.00025,0.00028,0.03564,0.316,0.02002,0.02042,0.02589,0.06005,0.00531,24.058
1,0.00202,1.33E-05,0.00051,0.00065,0.00152,0.0602,0.526,0.03478,0.03486,0.04316,0.10433,0.013293,20.883
2,0.00129,8.76E-06,0.00026,0.00058,0.00077,0.11166,1.002,0.06565,0.07022,0.08624,0.19695,0.015301,19.645
3,0.00096,4.61E-06,0.00023,0.00053,0.00068,0.05472,0.507,0.0319,0.03241,0.03717,0.09571,0.018637,17.875
4,0.00143,6.97E-06,0.00028,0.00066,0.00083,0.04211,0.385,0.02033,0.02555,0.03432,0.06099,0.015731,18.723


In [215]:
columns = [
    "Subject id",
    "Jitter (local)", "Jitter (local, absolute)", "Jitter (rap)", "Jitter (ppq5)", "Jitter (ddp)",  # Features 1-5
    "Shimmer (local)", "Shimmer (local, dB)", "Shimmer (apq3)", "Shimmer (apq5)", "Shimmer (apq11)", "Shimmer (dda)",  # Features 6-11
    "AC", "NTH", "HTN",  # Features 12-14
    "Median pitch", "Mean pitch", "Standard deviation", "Minimum pitch", "Maximum pitch",  # Features 15-19
    "Number of pulses", "Number of periods", "Mean period", "Standard deviation of period",  # Features 20-23
    "Fraction of locally unvoiced frames", "Number of voice breaks", "Degree of voice breaks",  # Features 24-26
    "UPDRS",  # Column 28
    "class"  ]
dataset = pd.read_csv("./data/train_data_1_.csv",names=columns)
dataset["class"] = pd.to_numeric(dataset["class"], errors='coerce').fillna(0).astype(int)
df4 = dataset[dataset["class"] == 0]
df4 = df4.rename(columns={
    "Jitter (local)": "MDVP:Jitter(%)",
    "Jitter (local, absolute)": "MDVP:Jitter(Abs)",
    "Jitter (rap)": "MDVP:RAP",
    "Jitter (ppq5)": "MDVP:PPQ",
    "Jitter (ddp)": "Jitter:DDP",
    "Shimmer (local)": "MDVP:Shimmer",
    "Shimmer (local, dB)": "MDVP:Shimmer(dB)",
    "Shimmer (apq3)": "Shimmer:APQ3",
    "Shimmer (apq5)": "Shimmer:APQ5",
    "Shimmer (apq11)": "MDVP:APQ",
    "Shimmer (dda)": "Shimmer:DDA",
    "AC": "NHR",
    "HTN": "HNR"
})

ordered_columns = [
    "MDVP:Jitter(%)", "MDVP:Jitter(Abs)", "MDVP:RAP", "MDVP:PPQ", "Jitter:DDP",
    "MDVP:Shimmer", "MDVP:Shimmer(dB)", "Shimmer:APQ3", "Shimmer:APQ5",
    "MDVP:APQ", "Shimmer:DDA", "NHR", "HNR"
]
df4 = df4[ordered_columns]
df_result = pd.concat([df2,df3,df4],axis=0, ignore_index=True)
df_result["status"] = 0
df_total = pd.concat([df_result,df], ignore_index=True)
df4.head()
df_total.to_csv('data/totalData.csv', index=False)
df_total.head()



,MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status
0,0.00289,0.00001,0.00166,0.00168,0.00498,0.01098,0.09700,0.00563,0.00680,0.00802,0.01689,0.00339,26.77500,0
1,0.00241,0.00001,0.00134,0.00138,0.00402,0.01015,0.08900,0.00504,0.00641,0.00762,0.01513,0.00167,30.94000,0
2,0.00212,0.00001,0.00113,0.00135,0.00339,0.01263,0.11100,0.00640,0.00825,0.00951,0.01919,0.00119,30.77500,0
3,0.00180,0.000009,0.00093,0.00107,0.00278,0.00954,0.08500,0.00469,0.00606,0.00719,0.01407,0.00072,32.68400,0
4,0.00178,0.000009,0.00094,0.00106,0.00283,0.00958,0.08500,0.00468,0.00610,0.00726,0.01403,0.00065,33.04700,0


## Concatenar los resultados de usuarios sanos

## Caracteristicas y Estado (Parkinson/Sano)

### Balanceo de datos

In [216]:
X = df_total.iloc[:,:-1]
y = df_total['status']
sm = SMOTE(random_state=300)
X, y = sm.fit_resample(X,y)
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)
X_features = pd.DataFrame(X, columns=df_total.columns[:-1])
Y_labels = y
joblib.dump(scaler, './app/export/scaler.pkl')

['./app/export/scaler.pkl']

## Entrenamiento

In [217]:
validation_size = 0.2

seed = 20

X_train, X_validation, Y_train, Y_validation = train_test_split(X_features,Y_labels, test_size=validation_size, random_state=seed)
num_instances = len(X_train)

scoring = 'accuracy'
results = []
# rfc = RandomForestClassifier()
# rfc.fit(X_train, Y_train)

svc = SVC(probability=True)
knn = KNeighborsClassifier()
rfc = RandomForestClassifier()
xgb = XGBClassifier()

svc_param_grid = {'C': [0.1, 1, 10]}
knn_param_grid = {'n_neighbors': [3, 5, 7]}
rfc_param_grid = {
    'n_estimators': [50, 100, 150],   # Número de árboles
    'max_depth': [3, 5, 7],           # Profundidad máxima de cada árbol
    'min_samples_split': [2, 5, 10]   # Mínimo de muestras para dividir un nodo
}
xgb_param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2]}


# xgb.fit(X_train, Y_train)

# svmc = SVC(kernel='linear', probability=True)
# svmc.fit(X_train, Y_train)

# knnc = KNeighborsClassifier(n_neighbors=5)
# knnc.fit(X_train, Y_train)


# eclf = VotingClassifier(estimators=[('rf', rfc),('knn',knnc),('svc', svmc)], voting='soft')
# eclf = VotingClassifier(estimators=[('gbc', gbc),('knn',knnc),('svc', svmc)], voting='hard')


# Ajustar SVC
svc_grid = GridSearchCV(estimator=svc, param_grid=svc_param_grid, cv=3, scoring='accuracy')
svc_grid.fit(X_train, Y_train)
best_svc = svc_grid.best_estimator_

# Ajustar KNN
knn_grid = GridSearchCV(estimator=knn, param_grid=knn_param_grid, cv=3, scoring='accuracy')
knn_grid.fit(X_train, Y_train)
best_knn = knn_grid.best_estimator_

# Ajustar XGBoost
rfc_grid = GridSearchCV(estimator=rfc, param_grid=rfc_param_grid, cv=3, scoring='accuracy')
rfc_grid.fit(X_train, Y_train)
best_rfc = rfc_grid.best_estimator_

xgb_grid = GridSearchCV(estimator=xgb, param_grid=xgb_param_grid, cv=3, scoring='accuracy')
xgb_grid.fit(X_train, Y_train)
best_xgb = xgb_grid.best_estimator_

eclf = VotingClassifier(estimators=[
    ('svc', best_svc),
    ('knn', best_knn),
    ('rfc', best_rfc),
    ('xgb', best_xgb)
], voting='soft')
eclf.fit(X_train, Y_train)

# eclf.fit(X_train, Y_train)
predictions = eclf.predict(X_validation)
cm = confusion_matrix(Y_validation, predictions)

TN, FP, FN, TP = cm.ravel()
report = classification_report(Y_validation, predictions, output_dict=True)

exactitud = accuracy_score(Y_validation, predictions)
sensibilidad = TP / (TP + FN)
especificidad = TN / (TN + FP)
precision = precision_score(Y_validation, predictions)
f1 = f1_score(Y_validation, predictions)

print(f"Precisión: {precision:.2f}")
print(f"Sensibilidad: {sensibilidad:.2f}")
print(f"Puntuación F1: {f1:.2f}")
print(f"Exactitud: {exactitud:.2f}")
print(f"Especificidad: {especificidad:.2f}")

joblib.dump(eclf, './app/export/modelo.pkl')

Precisión: 0.94
Sensibilidad: 1.00
Puntuación F1: 0.97
Exactitud: 0.97
Especificidad: 0.94


['./app/export/modelo.pkl']

## Obtener caracteristicas

In [219]:
def measurePitch(sound, unit, startTime, endTime,f0min,f0max):

    pitch = call(sound, "To Pitch", float(startTime), 75, 300) 

    meanF0 = call(pitch, "Get mean", startTime, endTime, unit) 

    maxf0 = call(pitch, "Get maximum", startTime, endTime, unit, "Parabolic")

    minf0 = call(pitch, "Get minimum", startTime, endTime, unit, "Parabolic")

    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)

    localJitter = call(pointProcess, "Get jitter (local)", startTime, endTime, 0.0001, 0.02, 1.3)

    pulses = call([sound, pitch], "To PointProcess (cc)")
    voice_report = call([sound, pitch, pulses], "Voice report", startTime, endTime, f0min, f0max, 1.3, 1.6, 0.03, 0.45)
    voice_report_array=re.findall(r'-?\d+\.?\d*',voice_report)
    hnr = voice_report_array[-1]
    nhr = voice_report_array[-2]
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", startTime, endTime, 0.0001, 0.02, 1.3)

    rapJitter = call(pointProcess, "Get jitter (rap)", startTime, endTime, 0.0001, 0.02, 1.3)

    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", startTime, endTime, 0.0001, 0.02, 1.3)

    ddpJitter = call(pointProcess, "Get jitter (ddp)", startTime, endTime, 0.0001, 0.02, 1.3)

    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", startTime, endTime, 0.0001, 0.02, 1.3, 1.6)

    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", startTime, endTime, 0.0001, 0.02, 1.3, 1.6)

    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", startTime, endTime, 0.0001, 0.02, 1.3, 1.6)

    apq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", startTime, endTime, 0.0001, 0.02, 1.3, 1.6)

    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", startTime, endTime, 0.0001, 0.02, 1.3, 1.6)

    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", startTime, endTime, 0.0001, 0.02, 1.3, 1.6)

    return localJitter,localabsoluteJitter, rapJitter, ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer, apq5Shimmer, apq11Shimmer,ddaShimmer, nhr, hnr

## Obtener caracteristicas de un solo archivo de audio

In [220]:
sound_file = "./grabaciones/Eva_07-11-24-14_51_37.wav"

sound = parselmouth.Sound(sound_file)
# sound = noiseReduce(sound_file)
f0min = 75
f0max = 300
startTime = 0
endTime = 0
unit = "Hertz"


(localJitter,localabsoluteJitter, rapJitter, ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer, apq5Shimmer, apq11Shimmer,ddaShimmer, nhr, hnr) = measurePitch(
    sound, unit, startTime, endTime, f0min, f0max)


## Prediccion de la enfermedad mediante modelo entrenado

In [221]:
dataframe = pd.DataFrame(np.column_stack([localJitter,localabsoluteJitter, rapJitter, ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer, apq5Shimmer, apq11Shimmer,ddaShimmer, nhr, hnr]), 
                  columns=["MDVP:Jitter(%)","MDVP:Jitter(Abs)","MDVP:RAP","MDVP:PPQ","Jitter:DDP","MDVP:Shimmer","MDVP:Shimmer(dB)","Shimmer:APQ3","Shimmer:APQ5","MDVP:APQ","Shimmer:DDA","NHR","HNR"])
dataframe = dataframe.apply(pd.to_numeric, errors="coerce")
dataframe.head()
# dataframe.to_csv('./data/dataset2.csv', index=False)
model = joblib.load("./app/export/modelo.pkl")
scaler = joblib.load("./app/export/scaler.pkl")
scaled_new_data = pd.DataFrame(scaler.transform(dataframe), columns=dataframe.columns)
prediction = model.predict(scaled_new_data)
if (prediction[0] == 1):
    print("Parkinson")
else:
    print("No Parkinson")

Parkinson


## Obtener caracteristicas de las grabaciones

In [101]:
directory = "./grabaciones"
data_list = []
f0min = 75
f0max = 300
startTime = 0
endTime = 0
unit = "Hertz"
for filename in os.listdir(directory):
  if filename.endswith(".wav"):
    sound_file = os.path.join(directory, filename)
    # Procesar el archivo de sonido
    sound = parselmouth.Sound(sound_file)
    # Extraer características
    (meanF0, maxf0, minf0, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter,
      localShimmer, localdbShimmer, apq3Shimmer, apq5Shimmer, apq11Shimmer, ddaShimmer, nhr, hnr) = measurePitch(
        sound, unit, startTime, endTime, f0min, f0max)
    # Agregar las características a la lista
    data_list.append([filename, meanF0, maxf0, minf0, localJitter, localabsoluteJitter, rapJitter,
                      ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, apq5Shimmer,
                      apq11Shimmer, ddaShimmer, nhr, hnr])

columns = ["Filename", "MDVP:Fo(Hz)", "MDVP:Fhi(Hz)", "MDVP:Flo(Hz)", "MDVP:Jitter(%)", "MDVP:Jitter(Abs)",
           "MDVP:RAP", "MDVP:PPQ", "Jitter:DDP", "MDVP:Shimmer", "MDVP:Shimmer(dB)",
           "Shimmer:APQ3", "Shimmer:APQ5", "MDVP:APQ", "Shimmer:DDA", "NHR", "HNR"]

dataframe = pd.DataFrame(data_list, columns=columns)
dataframe.to_csv("data/extracted_features.csv", index=False)
